In [2]:
from functions import *
import pandas as pd
import seaborn as sns
import statistics
from matplotlib import pyplot as plt
%matplotlib inline

blocks = pd.read_csv("./raw_data/pr/data_block.csv", names=["BLOCK", "NS"])
ecdsa_checks = pd.read_csv("./raw_data/pr/data_ecdsa.csv", names=["BLOCK", "NS"])
shielded_spend = pd.read_csv("./raw_data/pr/data_shieldedSpend.csv", names=["BLOCK", "NS"])
shielded_output = pd.read_csv("./raw_data/pr/data_shieldedOutput.csv", names=["BLOCK", "NS"])

mall_hash = pd.read_csv("./raw_data/pr/data_malleability_hash.csv", names=["BLOCK", "NS"])
mall_bindingSig = pd.read_csv("./raw_data/pr/data_malleability_bindingSig.csv", names=["BLOCK", "NS"])

In [5]:
#ecdsa_checks.tail()

BLOCK
0000000003fd2189b54eb5372f7b71116f4a547efcbe37bcf02cd90c6660ac68             [9548982]
0000000004017a587ee4ea74ced975ca03a21842fa0778234f7338deaa577626            [10066440]
00000000040a82098d9625573ee3ea9e6f6f5dc7e52c7852660157df2c1d4ec0        [103, 9218882]
00000000040b8efac072de5f701ac4d590c94e427dc6c1bc609f8a14ea3aaf99    [9449920, 9061849]
0000000004621bda72023d7a1266a2ae8b213703beb2abaa3ac77268a308b7bb             [9400644]
Name: NS, dtype: object

In [6]:
block_inputs = group_by_block(ecdsa_checks)
block_spends = group_by_block(shielded_spend)
block_outputs = group_by_block(shielded_output)
block_mall = group_by_block(mall_hash)
block_bindingSig = group_by_block(mall_bindingSig)

KeyboardInterrupt: 

In [ ]:
#block_inputs.loc['000000000074009a7fae00e0b3350ff3c05ff2c10597398578e361f8bd12b3d4']

In [ ]:
avg_input = median_percent_of_block(block_inputs, blocks)
avg_spend = median_percent_of_block(block_spends, blocks)
avg_output = median_percent_of_block(block_outputs, blocks)
avg_mall = median_percent_of_block(block_mall, blocks)
avg_bindingSig = median_percent_of_block(block_bindingSig, blocks)

In [ ]:
avg_input.describe()


In [ ]:
avg_spend.describe()

In [ ]:
avg_output.describe()

In [ ]:
avg_mall.describe()

In [ ]:
avg_bindingSig.describe()

In [ ]:
total_input = total_percent_of_block([block_inputs], blocks)
total_spend = total_percent_of_block([block_spends], blocks)
total_output = total_percent_of_block([block_outputs], blocks)
total_mall_sapling = total_percent_of_block([block_mall, block_bindingSig], blocks)
total = total_percent_of_block([block_bindingSig, block_mall, block_inputs, block_spends, block_outputs], blocks)

In [ ]:
total_input.describe()

In [ ]:
total_spend.describe()

In [ ]:
total_output.describe()

In [ ]:
total_mall_sapling.describe()

In [ ]:
total.describe()